In [6]:
from glob import glob
import json
import pandas as pd
import geopandas as gpd
import geohash
import shapely
from shapely.ops import cascaded_union
from shapely.geometry import Polygon, mapping

# import
import warnings
warnings.filterwarnings('ignore')
assets = '/app/assets/'

In [10]:
def obtener_geohashes_expandidos(poligono, precision, expansion=0.0005):
    # Expande el polígono ligeramente para asegurar cobertura completa
    poligono_expandido = poligono.buffer(expansion)
    
    # Convertir el polígono expandido a un formato GeoJSON
    geojson_expandido = mapping(poligono_expandido)
    
    # Calcular el bounding box del polígono expandido
    min_x, min_y, max_x, max_y = poligono_expandido.bounds
    
    # Calcular geohashes iniciales que cubren el bounding box
    hash_suroeste = geohash.encode(min_y, min_x, precision)
    hash_noreste = geohash.encode(max_y, max_x, precision)
    
    # Encuentra geohashes vecinos para cubrir completamente el área
    geohashes = set()
    geohashes.add(hash_suroeste)
    
    current_hash = hash_suroeste
    while current_hash != hash_noreste:
        # Añadir vecinos al este y norte
        vecinos = geohash.neighbors(current_hash)
        vecino_este = vecinos[2]  # Este
        vecino_norte = vecinos[1]  # Norte
        
        geohashes.add(vecino_este)
        geohashes.add(vecino_norte)
        
        # Moverse diagonalmente hacia el noreste para el siguiente ciclo
        current_hash = geohash.encode(
            geohash.decode(vecino_norte)[0],  # Latitud del vecino norte
            geohash.decode(vecino_este)[1],   # Longitud del vecino este
            precision
        )
    
    # Filtrar geohashes por aquellos dentro del polígono
    geohashes_dentro = [
        g for g in geohashes
        if poligono.contains(shapely.geometry.Point(geohash.decode(g)))
    ]
    
    return geohashes_dentro


def geohash_a_polygon(codigo_geohash):
    # Obtener el bounding box del geohash y convertirlo en un polígono
    puntos_geohash = geohash.bbox(codigo_geohash)
    poligono_geohash = Polygon([
        (puntos_geohash['w'], puntos_geohash['s']),
        (puntos_geohash['w'], puntos_geohash['n']),
        (puntos_geohash['e'], puntos_geohash['n']),
        (puntos_geohash['e'], puntos_geohash['s']),
        (puntos_geohash['w'], puntos_geohash['s'])
    ])
    return poligono_geohash

def geohash_a_gdf(codigo_geohash):
    poligono_geohash = geohash_a_polygon(codigo_geohash)
    # Crear un GeoDataFrame con el código geohash y el polígono
    geodf = gpd.GeoDataFrame({'geohash': [codigo_geohash], 'geometry': [poligono_geohash]})
    geodf = geodf.set_crs(4326)
    return geodf

def punto_a_geodf(x, y, precision=6):
    """
    Convierte un punto dado por coordenadas (x, y) en un GeoDataFrame que incluye
    el código geohash del punto y el polígono correspondiente al geohash.
    
    Parámetros:
    - x: Longitud del punto.
    - y: Latitud del punto.
    - precision: Precisión del geohash (número de caracteres del geohash).
    
    Retorna:
    - GeoDataFrame con dos columnas: 'geohash' y 'geometry', donde 'geometry'
      contiene el polígono correspondiente al geohash.
    """
    # Calcular el geohash para el punto dado
    codigo_geohash = geohash.encode(y, x, precision)
    
    geodf = geohash_a_gdf(codigo_geohash)
    
    return geodf

def expandir_geohash_centroide_niveles(codigo_geohash, niveles=1):
    """
    Expande desde un geohash central para cubrir un área cuadrada alrededor del
    centroide utilizando geohashes adyacentes a múltiples niveles.
    
    Parámetros:
    - codigo_geohash: Código geohash del centroide.
    - niveles: Número de niveles de vecinos a expandir.
    
    Retorna:
    - GeoDataFrame con las columnas 'geohash' y 'geometry', donde 'geometry'
      contiene los polígonos correspondientes a los geohashes.
    """
    
    def obtener_vecinos(ghash):
        """Obtiene todos los vecinos directos de un geohash dado."""
        return geohash.neighbors(ghash)
    
    # Inicializar la lista de geohashes con el central
    geohashes = {codigo_geohash}
    geohashes_procesar = {codigo_geohash}
    
    for _ in range(niveles):
        nuevos_geohashes = set()
        for ghash in geohashes_procesar:
            vecinos = obtener_vecinos(ghash)
            nuevos_geohashes.update(vecinos)
        geohashes_procesar = nuevos_geohashes - geohashes  # Procesar solo nuevos geohashes
        geohashes.update(nuevos_geohashes)
    
    # Crear polígonos para cada geohash y agregarlos a una lista
    poligonos = []
    for gh in geohashes:
        puntos = geohash.bbox(gh)
        poligono = Polygon([
            (puntos['w'], puntos['s']),
            (puntos['w'], puntos['n']),
            (puntos['e'], puntos['n']),
            (puntos['e'], puntos['s']),
            (puntos['w'], puntos['s'])
        ])
        poligonos.append(poligono)
    
    # Crear GeoDataFrame
    geodf = gpd.GeoDataFrame({'geohash': list(geohashes), 'geometry': poligonos})
    geodf = geodf.set_crs("EPSG:4326")
    
    return geodf

def get_geohash_set(area, precision, step):
    gdf_input = area.copy()
    gdf_input = gdf_input.to_crs(4326)
    gdf_input['centroid'] = gdf_input['geometry'].centroid
    area_interes = gdf_input['geometry'][0].area

    centroide_lat, centroide_lon = gdf_input['centroid'][0].y, gdf_input['centroid'][0].x
    geodf_with_hash = punto_a_geodf(x=centroide_lon, y=centroide_lat, precision=precision)
    codigo_geohash = geodf_with_hash['geohash'][0]
    niveles = 2

    while(True):
        extended_geohash = expandir_geohash_centroide_niveles(codigo_geohash, niveles=niveles)
        extended_geohash = extended_geohash.to_crs(gdf_input.crs)
        overlay_geohash = gpd.overlay(gdf_input, extended_geohash)
        overlay_area = overlay_geohash['geometry'].area.sum()
        if(overlay_area < area_interes):
            niveles += step
        else:
            break
        # print(overlay_geohash.shape[0])
    return_geohash = gpd.sjoin(extended_geohash, gdf_input)
    return_geohash = return_geohash.to_crs(4326)
    return return_geohash, niveles

In [11]:
area = gpd.read_parquet(f'{assets}/area_scope/area_scope.parquet')

In [24]:
precision = 8
step = 100

geohashes, _ = get_geohash_set(area, precision=precision, step=step)
geohashes = geohashes.to_crs(area.crs)

geohashes_cols = ['geohash', 'geometry']
geohashes = geohashes[geohashes_cols]

geohashes.to_parquet(f'/app/data/set/gh/level_{precision}.parquet')